In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

In [2]:
# Загрузка данных
df = pd.read_csv("train.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
# Удаление столбца PassengerId
df.drop(columns=['PassengerId'], inplace=True)

In [4]:
# Преобразование пола в числовой формат
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

In [ ]:
# Определение пропущенных значений
missing_values = df.isnull().sum()
missing_values = missing_values[missing_values > 0]

# Вывод результатов
print("Пропущенные значения в данных:")
print(missing_values)


In [6]:
# Вручную заполняем пропущенные значения
df.loc[df['Name'].str.contains('Icard, Miss. Amelie'), 'Embarked'] = 'C'  # Шербур
df.loc[df['Name'].str.contains('Stone, Mrs. George Nelson'), 'Embarked'] = 'S'  # Саутгемптон

In [7]:
# Создаём объект KNNImputer
imputer = KNNImputer(n_neighbors=5)

# Применяем imputer к данным
df[['Age']] = imputer.fit_transform(df[['Age']])

In [8]:
# Подсчет частоты значений в Embarked
embarked_counts = df['Embarked'].value_counts()

# Преобразование Embarked в числовой формат на основе частотности
df['Embarked'] = df['Embarked'].map(embarked_counts)

In [ ]:
df.info()

In [10]:
# Добавление столбца количества занимаемых кают
df['Cabin_Count'] = df['Cabin'].apply(lambda x: len(str(x).split()) if pd.notna(x) else 1)

In [11]:
# # Фильтрация кают, содержащих только буквы
# cabin_letters_only = df[df['Cabin'].notna() & df['Cabin'].str.match(r'^[A-Za-z]+$', na=False)]

# # Вывод столбцов Name и Cabin
# print(cabin_letters_only[['Name', 'Cabin']])

In [12]:
# Извлечение только первой буквы каюты
df['Cabin_Letter'] = df['Cabin'].str.extract('([A-Za-z])')

# Удаление столбца Cabin
df.drop(columns=['Cabin'], inplace=True)

In [13]:
# Извлечем номер каюты и закондируем

In [14]:
# Извлечение титула (обращения) из имени
df['Title'] = df['Name'].str.extract(r' ([A-Za-z]+)\.')

# Извлечение фамилии
df['Surname'] = df['Name'].str.split(',').str[0]

# Удаление столбца Name
df = df.drop(columns=['Name'])

In [15]:
# Кодирование Title
df['Title'] = df['Title'].astype('category').cat.codes

# Кодирование Surname
df['Surname'] = df['Surname'].astype('category').cat.codes

In [ ]:
# Заполняем пропущенные значения "0" (создаем новую категорию)
df['Cabin_Letter'] = df['Cabin_Letter'].fillna('0')

# One-Hot Encoding для Cabin_Letter
df = pd.get_dummies(df, columns=['Cabin_Letter'], prefix='Cabin', dtype=int)

# Проверяем результат
print(df.head())

# Убеждаемся, что исходный столбец удален
if 'Cabin_Letter' in df.columns:
    df.drop(columns=['Cabin_Letter'], inplace=True)

In [17]:
# Создание новых признаков
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)
df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 12, 18, 35, 60, 100], labels=[0, 1, 2, 3, 4]).astype(float)
df['FarePerPerson'] = df['Fare'] / df['FamilySize']

In [ ]:
# Выбираем признаки для обучения модели
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 
            'Cabin_Count', 'Title', 'Surname', 'Cabin_A', 'Cabin_B', 'Cabin_C', 
            'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_T',
            'FamilySize', 'IsAlone', 'AgeGroup', 'FarePerPerson']

# Удаляем строки, где `Cabin_0` = NaN (если такие есть)
df = df.dropna(subset=['Cabin_0'])

# Разделяем данные на обучающие и тестовые
X = df[features]
y = df['Cabin_0']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучаем модель RandomForest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Делаем предсказание на тестовых данных
y_pred = model.predict(X_test)

# Оцениваем точность модели
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy:.4f}')

# Заполняем пропуски в `Cabin_0` с помощью обученной модели
df.loc[df['Cabin_0'] == 0, 'Cabin_0'] = model.predict(df.loc[df['Cabin_0'] == 0, features])

In [ ]:
# Выбираем признаки для обучения модели
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 
            'Cabin_Count', 'Title', 'Surname', 'Cabin_0', 'Cabin_A', 'Cabin_B', 
            'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_T']

X = df[features]  # Данные (признаки)
y = df['Survived']  # Целевая переменная

# Разделяем данные на обучающую и тестовую выборки (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Определяем модели
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Random Forest": RandomForestClassifier(n_estimators=200, random_state=42),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=200, random_state=42),
    "Extra Trees": ExtraTreesClassifier(n_estimators=200, random_state=42),
    "XGBoost": xgb.XGBClassifier(n_estimators=200, random_state=42, use_label_encoder=False, eval_metric='logloss'),
    "LightGBM": lgb.LGBMClassifier(n_estimators=200, random_state=42),
    "CatBoost": CatBoostClassifier(iterations=200, random_state=42, verbose=0)
}

# Обучаем и тестируем модели
results = {}

for name, model in models.items():
    try:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        results[name] = accuracy  # Сохраняем точность

        print(f"\n🔹 {name}")
        print(f"Точность: {accuracy:.4f}")
        print(classification_report(y_test, y_pred))
    except Exception as e:
        print(f"\n❌ Ошибка в {name}: {e}")

# Сортируем модели по убыванию точности
sorted_results = sorted(results.items(), key=lambda x: x[1], reverse=True)

# Выводим лучшую модель
best_model_name, best_accuracy = sorted_results[0]
print(f"\n✅ Лучшая модель: {best_model_name} с точностью {best_accuracy:.4f}")

# Вывод всех моделей в порядке точности
print("\n📊 Точность моделей (от лучшей к худшей):")
for name, acc in sorted_results:
    print(f"{name}: {acc:.4f}")


In [ ]:
# Загрузка test.csv с сохранением PassengerId
test_df = pd.read_csv("test.csv")

# Сохраняем PassengerId для выходного файла
passenger_ids = test_df['PassengerId']

# Применяем те же преобразования к test_df
test_df.drop(columns=['PassengerId', 'Ticket'], inplace=True)
test_df['Sex'] = test_df['Sex'].map({'male': 0, 'female': 1})
test_df['Embarked'] = test_df['Embarked'].map(embarked_counts)
test_df[['Age']] = imputer.transform(test_df[['Age']])
test_df['Cabin_Count'] = test_df['Cabin'].apply(lambda x: len(str(x).split()) if pd.notna(x) else 1)
test_df['Cabin_Letter'] = test_df['Cabin'].str.extract('([A-Za-z])')
test_df.drop(columns=['Cabin'], inplace=True)
test_df['Title'] = test_df['Name'].str.extract(r' ([A-Za-z]+)\.')
test_df['Surname'] = test_df['Name'].str.split(',').str[0]
test_df.drop(columns=['Name'], inplace=True)
test_df['Title'] = test_df['Title'].astype('category').cat.codes
test_df['Surname'] = test_df['Surname'].astype('category').cat.codes
test_df['Cabin_Letter'] = test_df['Cabin_Letter'].fillna('0')
test_df = pd.get_dummies(test_df, columns=['Cabin_Letter'], prefix='Cabin', dtype=int)

# Добавляем недостающие столбцы, если они есть
for col in df.columns:
    if col not in test_df.columns:
        test_df[col] = 0  

# Убеждаемся, что порядок столбцов совпадает
test_df = test_df[X.columns]

# Предсказание выживания лучшей моделью (Random Forest / CatBoost / XGBoost / LightGBM)
best_model = model  # Замените на лучшую модель, например: best_model = catboost_model
test_df['Survived'] = best_model.predict(test_df)

# Создание выходного файла gender_submission.csv
submission = pd.DataFrame({'PassengerId': passenger_ids, 'Survived': test_df['Survived']})
submission.to_csv("gender_submission.csv", index=False)

print("\n✅ Файл gender_submission.csv успешно сохранен!")


